In [0]:
import pyspark
from pyspark.sql.functions import *

In [0]:
dbutils.secrets.listScopes()

[SecretScope(name='healthcarescope')]

In [0]:
dbutils.secrets.list(scope = 'healthcarescope')

[SecretMetadata(key='blobaccesskey')]

In [0]:

secret_name = dbutils.secrets.get(scope = "healthcarescope", key = "blobaccesskey")

In [0]:
print(secret_name)

[REDACTED]


In [0]:
spark.conf.set(
    "fs.azure.account.key.healthcareprojectblob.dfs.core.windows.net",
    dbutils.secrets.get(scope = "healthcarescope", key = "blobaccesskey"))


In [0]:
display(dbutils.fs.ls("abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1725312200000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/claims.csv,claims.csv,5766,1725461306000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/disease.csv,disease.csv,1489,1725312200000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/group.csv,group.csv,4390,1725312200000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/hospital.csv,hospital.csv,1328,1725312200000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1725312200000
abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1725312201000


In [0]:
# # Mount the Azure Blob Storage container as a DBFS path
# dbutils.fs.mount(
#   source="wasbs://rawhealthdata@healthcareprojectblob.blob.core.windows.net",
#   mount_point="/mnt/rawhealthdata",
#   extra_configs={
#     "fs.azure.account.key.healthcareprojectblob.blob.core.windows.net": "0izVsgjt8yavZcTHY3jKLiEiVe0Nu7jqnlYlYXCi3eAF/WsbNH1eY0Cvvd59kRgOIpunsgWDknLx+ASteulWxw=="
#   }
# )

# # List the files in the mounted DBFS path
# dbutils.fs.ls("/mnt/rawhealthdata")

In [0]:
# Unmount the Azure Blob Storage container
# dbutils.fs.unmount("/mnt/rawhealthdata")

In [0]:
data = spark.read.csv("abfss://rawhealthdata@healthcareprojectblob.dfs.core.windows.net/hospital.csv", header=True, inferSchema=True)

In [0]:
display(data)

Hospital_id,Hospital_name,city,state,country
H1000,All India Institute of Medical Sciences,New Delhi,NaN,India
H1001,Medanta The Medicity,Gurgaon,Haryana,India
H1002,The Christian Medical College,Vellore,Tamil Nadu,India
H1003,PGIMER - Postgraduate Institute of Medical Education and Research,Chandigarh,Haryana,India
H1004,Apollo Hospital - Chennai,Chennai,Tamil Nadu,India
H1005,P. D. Hinduja National Hospital & Medical Research Centre,Mumbai,Maharashtra,India
H1006,Breach Candy Hospital,Mumbai,Maharashtra,India
H1007,Fortis Flt. Lt. Rajan Dhall Hospital,New Delhi,NaN,India
H1008,King Edward Memorial Hospital,Mumbai,Maharashtra,India
H1009,Indraprastha Apollo Hospital,Delhi,NaN,India


In [0]:
# to do - clean state and city

In [0]:
data.columns

['Hospital_id', 'Hospital_name', 'city', 'state', 'country']

In [0]:
data.groupby(['Hospital_id', 'Hospital_name', 'city', 'state', 'country']).count().where("count > 1").show(15, False)

+-----------+-------------+----+-----+-------+-----+
|Hospital_id|Hospital_name|city|state|country|count|
+-----------+-------------+----+-----+-------+-----+
+-----------+-------------+----+-----+-------+-----+



In [0]:
# check null value for all the columns
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+-----------+-------------+----+-----+-------+
|Hospital_id|Hospital_name|city|state|country|
+-----------+-------------+----+-----+-------+
|          0|            0|   0|    4|      0|
+-----------+-------------+----+-----+-------+



In [0]:
data = data.dropDuplicates()

In [0]:
data = data.fillna({"state" : "Union Territory"})

In [0]:
data.show()

+-----------+--------------------+----------+-----------+-------+
|Hospital_id|       Hospital_name|      city|      state|country|
+-----------+--------------------+----------+-----------+-------+
|      H1010|Lilavati Hospital...|    Mumbai|Maharashtra|  India|
|      H1017|   Manipal Hospitals| Bengaluru|  Karnataka|  India|
|      H1006|Breach Candy Hosp...|    Mumbai|Maharashtra|  India|
|      H1016|Jaslok Hospital a...|    Mumbai|Maharashtra|  India|
|      H1012|Bombay Hospital &...|    Mumbai|Maharashtra|  India|
|      H1018|Yashoda Hospital ...| Hyderabad|  Telangana|  India|
|      H1015|Fortis Hospital M...|    Mumbai|Maharashtra|  India|
|      H1009|Indraprastha Apol...|     Delhi|        NaN|  India|
|      H1014|Fortis Hiranandan...|    Mumbai|Maharashtra|  India|
|      H1004|Apollo Hospital -...|   Chennai| Tamil Nadu|  India|
|      H1002|The Christian Med...|   Vellore| Tamil Nadu|  India|
|      H1003|PGIMER - Postgrad...|Chandigarh|    Haryana|  India|
|      H10

In [0]:
data = data.replace("NaN", None)

In [0]:
data.show()

+-----------+--------------------+----------+-----------+-------+
|Hospital_id|       Hospital_name|      city|      state|country|
+-----------+--------------------+----------+-----------+-------+
|      H1010|Lilavati Hospital...|    Mumbai|Maharashtra|  India|
|      H1017|   Manipal Hospitals| Bengaluru|  Karnataka|  India|
|      H1006|Breach Candy Hosp...|    Mumbai|Maharashtra|  India|
|      H1016|Jaslok Hospital a...|    Mumbai|Maharashtra|  India|
|      H1012|Bombay Hospital &...|    Mumbai|Maharashtra|  India|
|      H1018|Yashoda Hospital ...| Hyderabad|  Telangana|  India|
|      H1015|Fortis Hospital M...|    Mumbai|Maharashtra|  India|
|      H1009|Indraprastha Apol...|     Delhi|       NULL|  India|
|      H1014|Fortis Hiranandan...|    Mumbai|Maharashtra|  India|
|      H1004|Apollo Hospital -...|   Chennai| Tamil Nadu|  India|
|      H1002|The Christian Med...|   Vellore| Tamil Nadu|  India|
|      H1003|PGIMER - Postgrad...|Chandigarh|    Haryana|  India|
|      H10

In [0]:
data = data.fillna({"state" : "Union Territory"})

In [0]:
data.show()

+-----------+--------------------+----------+---------------+-------+
|Hospital_id|       Hospital_name|      city|          state|country|
+-----------+--------------------+----------+---------------+-------+
|      H1010|Lilavati Hospital...|    Mumbai|    Maharashtra|  India|
|      H1017|   Manipal Hospitals| Bengaluru|      Karnataka|  India|
|      H1006|Breach Candy Hosp...|    Mumbai|    Maharashtra|  India|
|      H1016|Jaslok Hospital a...|    Mumbai|    Maharashtra|  India|
|      H1012|Bombay Hospital &...|    Mumbai|    Maharashtra|  India|
|      H1018|Yashoda Hospital ...| Hyderabad|      Telangana|  India|
|      H1015|Fortis Hospital M...|    Mumbai|    Maharashtra|  India|
|      H1009|Indraprastha Apol...|     Delhi|Union Territory|  India|
|      H1014|Fortis Hiranandan...|    Mumbai|    Maharashtra|  India|
|      H1004|Apollo Hospital -...|   Chennai|     Tamil Nadu|  India|
|      H1002|The Christian Med...|   Vellore|     Tamil Nadu|  India|
|      H1003|PGIMER 

In [0]:
# Replace Delhi with New Delhi
data = data.replace("Delhi", "New Delhi")

In [0]:
data.show()

+-----------+--------------------+----------+---------------+-------+
|Hospital_id|       Hospital_name|      city|          state|country|
+-----------+--------------------+----------+---------------+-------+
|      H1010|Lilavati Hospital...|    Mumbai|    Maharashtra|  India|
|      H1017|   Manipal Hospitals| Bengaluru|      Karnataka|  India|
|      H1006|Breach Candy Hosp...|    Mumbai|    Maharashtra|  India|
|      H1016|Jaslok Hospital a...|    Mumbai|    Maharashtra|  India|
|      H1012|Bombay Hospital &...|    Mumbai|    Maharashtra|  India|
|      H1018|Yashoda Hospital ...| Hyderabad|      Telangana|  India|
|      H1015|Fortis Hospital M...|    Mumbai|    Maharashtra|  India|
|      H1009|Indraprastha Apol...| New Delhi|Union Territory|  India|
|      H1014|Fortis Hiranandan...|    Mumbai|    Maharashtra|  India|
|      H1004|Apollo Hospital -...|   Chennai|     Tamil Nadu|  India|
|      H1002|The Christian Med...|   Vellore|     Tamil Nadu|  India|
|      H1003|PGIMER 

In [0]:
# # Now we need to write this data into stagging area for the next step.
# # boilerplate code (standard code for certain operation)

# Define the output staging path
output_stagging_path = "abfss://stagginglayerhealthdata@healthcareprojectblob.dfs.core.windows.net"

# Write the data to the staging area
data.coalesce(1).write.mode("append").format("com.databricks.spark.csv").option("header", "true").option("format", "csv").save(output_stagging_path)

# List all files in the output staging path
files = dbutils.fs.ls(output_stagging_path)

# Identify part files and non-part files
part_files = [x for x in files if x.name.startswith("part-")]
non_part_files = [x for x in files if x.name.startswith("_")]

# Move the part file to the desired location
if part_files:
    dbutils.fs.mv(part_files[0].path, f"{output_stagging_path}/hospitalstagging.csv")

# Remove non-part files
for file in non_part_files:
    dbutils.fs.rm(file.path)